In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.0-bin-hadoop3.2')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/23 15:04:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Reading csv

In [4]:
df = spark.read.csv('../data/Spark_DataFrames/sales_info.csv', 
                     inferSchema=True, 
                     header=True)

In [5]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



## Groupbys

In [10]:
# Sales by company
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [11]:
# All sales
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [12]:
# combined
df.groupBy('Company').agg({"Sales":'sum'}).show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



## Using pyspark functions

In [21]:
from pyspark.sql.functions import countDistinct, avg, stddev, format_number

In [25]:
df.select(
    [stddev('Sales').alias('std'), 
     avg('Sales').alias('avg')]
         ).show()

+------------------+-----------------+
|               std|              avg|
+------------------+-----------------+
|250.08742410799007|360.5833333333333|
+------------------+-----------------+



In [26]:
# make it prettier
df.select(
    [format_number(stddev('Sales'),2).alias('std'), 
     format_number(avg('Sales'),2).alias('avg')]
         ).show()

+------+------+
|   std|   avg|
+------+------+
|250.09|360.58|
+------+------+



### Ordering

In [27]:
df.orderBy('sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [30]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

